In [ ]:
at this code

import socket
import struct
import time
from python_detenv import load_env

load_env()

from collections import defaultdict
from influxdb_client import InfluxDBClient, Point, WriteOptions

# InfluxDB configuration
INFLUXDB_URL = os.getenv(INFLUXDB_URL)
INFLUXDB_TOKEN = os.getenv("INFLUXDB_TOKEN")
INFLUXDB_ORG = os.getenv("INFLUXDB_ORG")
INFLUXDB_BUCKET = os.getenv("INFLUXDB_BUCKET")

# PMU data packet format
PACKET_FORMAT = '>Id12f'  # PMU ID, Timestamp, 12 floats
PACKET_SIZE = struct.calcsize(PACKET_FORMAT)

# Expected PMU IDs
EXPECTED_PMUS = set(range(1, 10))  # PMU IDs from 1 to 9 (inclusive)

# Buffer to hold incoming data
data_buffer = defaultdict(dict)

def receive_full_packet(conn, size):
    """Receive exactly 'size' bytes from the TCP socket."""
    data = b''
    while len(data) < size:
        packet = conn.recv(size - len(data))
        if not packet:
            return None
        data += packet
    return data

def receive_full_line(conn):
    """Receive a full line ending with newline for non-compressed data."""
    line = b''
    while True:
        ch = conn.recv(1)
        if not ch:
            return None
        line += ch
        if ch == b'\n':
            break
    return line.decode('utf-8')

def send_compressed_in_batches(conn, addr, write_api):
    """Original method: receive compressed (struct) PMU data, buffer by timestamp, send batch when complete."""
    print(f"[Compressed/Batched] Connected to PMU at {addr}")
    try:
        while True:
            data = receive_full_packet(conn, PACKET_SIZE)
            if data is None:
                print(f"Connection closed by {addr}")
                break

            unpacked = struct.unpack(PACKET_FORMAT, data)
            pmu_id = int(unpacked[0])
            timestamp = unpacked[1]
            phasors = unpacked[2:]

            # Buffer per timestamp
            data_buffer[timestamp][pmu_id] = phasors

            # When all expected PMUs are in, send batch to InfluxDB
            if EXPECTED_PMUS.issubset(data_buffer[timestamp].keys()):
                for batch_pmu_id, phasor_data in data_buffer[timestamp].items():
                    point = Point("pmu_measurements").tag("pmu_id", batch_pmu_id).time(int(timestamp * 1e9))
                    labels = ['v_a_mag', 'v_a_ang', 'v_b_mag', 'v_b_ang', 'v_c_mag', 'v_c_ang',
                              'i_a_mag', 'i_a_ang', 'i_b_mag', 'i_b_ang', 'i_c_mag', 'i_c_ang']
                    for label, value in zip(labels, phasor_data):
                        point.field(label, value)
                    write_api.write(bucket=INFLUXDB_BUCKET, record=point)
                del data_buffer[timestamp]
    finally:
        conn.close()

def choose_experiment():
    print("\nChoose experiment mode:")
    print("2 - Real-time Batch (write every N PMU packets, struct compressed)")
    return choice

def handle_client_connection(conn, addr, write_api, mode):
    if mode == "1":
        send_compressed_in_batches(conn, addr, write_api)
    else:
        print("Invalid experiment mode selected.")
        conn.close()"

def handle_udp_connection(server_sock, write_api, mode):
    print("PDC is listening for **UDP** packets from PMUs...")
    while True:
        data, addr = server_sock.recvfrom(PACKET_SIZE)
        if not data:
            continue  # skip if empty

        if mode == "1":
            unpacked = struct.unpack(PACKET_FORMAT, data)
            pmu_id = int(unpacked[0])
            timestamp = unpacked[1]
            phasors = unpacked[2:]

            data_buffer[timestamp][pmu_id] = phasors
            if EXPECTED_PMUS.issubset(data_buffer[timestamp].keys()):
                for batch_pmu_id, phasor_data in data_buffer[timestamp].items():
                    point = Point("pmu_measurements").tag("pmu_id", batch_pmu_id).time(int(timestamp * 1e9))
                    labels = ['v_a_mag', 'v_a_ang', 'v_b_mag', 'v_b_ang', 'v_c_mag', 'v_c_ang',
                              'i_a_mag', 'i_a_ang', 'i_b_mag', 'i_b_ang', 'i_c_mag', 'i_c_ang']
                    for label, value in zip(labels, phasor_data):
                        point.field(label, value)
                    write_api.write(bucket=INFLUXDB_BUCKET, record=point)
                del data_buffer[timestamp]
        elif mode == "2":
            unpacked = struct.unpack(PACKET_FORMAT, data)
            pmu_id = int(unpacked[0])
            timestamp = unpacked[1]
            phasors = unpacked[2:]
            point = Point("pmu_measurements").tag("pmu_id", pmu_id).time(int(timestamp * 1e9))
            labels = ['v_a_mag', 'v_a_ang', 'v_b_mag', 'v_b_ang', 'v_c_mag', 'v_c_ang',
                      'i_a_mag', 'i_a_ang', 'i_b_mag', 'i_b_ang', 'i_c_mag', 'i_c_ang']
            for label, value in zip(labels, phasors):
                point.field(label, value)
            write_api.write(bucket=INFLUXDB_BUCKET, record=point)
        elif mode == "3":
            try:
                line = data.decode('utf-8')
                parts = line.strip().split(',')
                if len(parts) != 14:
                    print(f"Invalid data line: {line.strip()}")
                    continue
                pmu_id = int(parts[0])
                timestamp = float(parts[1])
                phasors = list(map(float, parts[2:]))
                point = Point("pmu_measurements").tag("pmu_id", pmu_id).time(int(timestamp * 1e9))
                labels = ['v_a_mag', 'v_a_ang', 'v_b_mag', 'v_b_ang', 'v_c_mag', 'v_c_ang',
                          'i_a_mag', 'i_a_ang', 'i_b_mag', 'i_b_ang', 'i_c_mag', 'i_c_ang']
                for label, value in zip(labels, phasors):
                    point.field(label, value)
                write_api.write(bucket=INFLUXDB_BUCKET, record=point)
            except Exception as e:
                print(f"Error processing UDP line: {line.strip()}, error: {e}")

def read_protocol_and_mode():
    # Default values if config file not found
    protocol, mode = None, None
    config_path = r"\\DESKTOP-JKQLLNS\Users\igee2025\Desktop\config.ini"
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            lines = f.readlines()
            if len(lines) >= 1:
                protocol = lines[0].strip().split("=")[1].strip().replace('"', "").replace("'", "")
            if len(lines) >= 2:
                mode = lines[1].strip().split("=")[1].strip().replace('"', "").replace("'", "")
    return protocol, mode

def main():
    protocol = choose_protocol()
    mode = choose_experiment()
    print(f"Running experiment mode: {mode}, Protocol: {protocol}")

    client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
    write_api = client.write_api(write_options=WriteOptions(batch_size=500, flush_interval=10000))

    if protocol == "TCP":
        server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_sock.bind(('0.0.0.0', 9009))
        server_sock.listen(5)
        print("PDC is listening for **TCP** connections from PMUs...")
        while True:
            conn, addr = server_sock.accept()
            handle_client_connection(conn, addr, write_api, mode)
    else:  # UDP
        server_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        server_sock.bind(('0.0.0.0', 9009))
        handle_udp_connection(server_sock, write_api, mode)

if __name__ == "__main__":
    main()

